<a href="https://colab.research.google.com/github/comojin1994/Deep_Learning_Study/blob/master/3step_lecture/Vanila_RNN_LSTM_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
tf.__version__

TensorFlow 2.x selected.


'2.1.0'

### Hyperparameter

In [0]:
EPOCHS = 10
NUM_WORDS = 10000

### RNN Model

In [0]:
class RNN(tf.keras.Model):
    def __init__(self):
        super(RNN, self).__init__()
        ### NUM_WORDS만큼의 one-hot vector를 16dim의 벡터로 변환
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        self.rnn = tf.keras.layers.SimpleRNN(32)
        self.dense = tf.keras.layers.Dense(2, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        x = self.rnn(x)
        return self.dense(x)

### LSTM Model

In [0]:
class LSTM(tf.keras.Model):
    def __init__(self):
        super(LSTM, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        self.lstm = tf.keras.layers.LSTM(32)
        self.dense = tf.keras.layers.Dense(2, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        x = self.lstm(x)
        return self.dense(x)

### GRU Model

In [0]:
class GRU(tf.keras.Model):
    def __init__(self):
        super(GRU, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        self.lstm = tf.keras.layers.GRU(32)
        self.dense = tf.keras.layers.Dense(2, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        x = self.lstm(x)
        return self.dense(x)

### Train, Test Loop

In [0]:
@tf.function
def train_step(model, inputs, labels, optimizer, loss_object, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(model, inputs, labels, loss_object, test_loss, test_accuracy):
    predictions = model(inputs, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

### Load Data

In [0]:
### imdb 데이터 : 영화 리뷰에 대한 감성 분석
### y가 0 / 1로 이루어져 있음
imdb = tf.keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)

### padding : 'post', 'pre'
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=32)

x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                       maxlen=32,
                                                       padding='pre',
                                                       value=0)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

### **Vanila RNN**

### Load Model, loss, optimizer, metric

In [0]:
rnn_model = RNN()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### Training

In [8]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(rnn_model, seqs, labels, optimizer, loss_object, train_loss, train_accuracy)

    for test_seqs, test_labels in test_ds:
        test_step(rnn_model, test_seqs, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {} Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(
          epoch + 1,
          train_loss.result(),
          train_accuracy.result()*100,
          test_loss.result(),
          test_accuracy.result()*100
    ))

    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1 Loss: 0.5461286306381226, Accuracy: 70.56800079345703, Test Loss: 0.46181321144104004, Test Accuracy: 78.28800201416016
Epoch 2 Loss: 0.360223650932312, Accuracy: 84.14800262451172, Test Loss: 0.4842202961444855, Test Accuracy: 77.83599853515625
Epoch 3 Loss: 0.22313256561756134, Accuracy: 91.34400177001953, Test Loss: 0.5924072861671448, Test Accuracy: 74.79199981689453
Epoch 4 Loss: 0.09801386296749115, Accuracy: 96.69200134277344, Test Loss: 0.7976824045181274, Test Accuracy: 73.79999542236328
Epoch 5 Loss: 0.04604891315102577, Accuracy: 98.54000091552734, Test Loss: 0.9864574670791626, Test Accuracy: 74.29199981689453
Epoch 6 Loss: 0.021950941532850266, Accuracy: 99.33200073242188, Test Loss: 1.1732544898986816, Test Accuracy: 74.14800262451172
Epoch 7 Loss: 0.015439605340361595, Accuracy: 99.52799987792969, Test Loss: 1.2891355752944946, Test Accuracy: 73.58000183105469
Epoch 8 Loss: 0.019586259499192238, Accuracy: 99.38399505615234, Test Loss: 1.3166275024414062, Test Acc

## **LSTM**

### Load Model, loss, optimizer, metrics

In [0]:
lstm_model = LSTM()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### Training

In [7]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(lstm_model, seqs, labels, optimizer, loss_object, train_loss, train_accuracy)

    for test_seqs, test_labels in test_ds:
        test_step(lstm_model, test_seqs, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {} Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(
          epoch + 1,
          train_loss.result(),
          train_accuracy.result()*100,
          test_loss.result(),
          test_accuracy.result()*100
    ))

    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1 Loss: 0.5028752088546753, Accuracy: 73.98399353027344, Test Loss: 0.43520388007164, Test Accuracy: 79.33599853515625
Epoch 2 Loss: 0.37079429626464844, Accuracy: 83.49600219726562, Test Loss: 0.4555682837963104, Test Accuracy: 78.91999816894531
Epoch 3 Loss: 0.31726810336112976, Accuracy: 86.3239974975586, Test Loss: 0.492877334356308, Test Accuracy: 78.2760009765625
Epoch 4 Loss: 0.2678675949573517, Accuracy: 88.8759994506836, Test Loss: 0.5092926621437073, Test Accuracy: 77.95999908447266
Epoch 5 Loss: 0.21883524954319, Accuracy: 91.2239990234375, Test Loss: 0.6740652918815613, Test Accuracy: 77.3759994506836
Epoch 6 Loss: 0.17508773505687714, Accuracy: 93.21600341796875, Test Loss: 0.6611217856407166, Test Accuracy: 76.73600006103516
Epoch 7 Loss: 0.1319689303636551, Accuracy: 94.85600280761719, Test Loss: 0.8110933303833008, Test Accuracy: 75.8759994506836
Epoch 8 Loss: 0.10157676041126251, Accuracy: 96.31999969482422, Test Loss: 0.9797196984291077, Test Accuracy: 75.704002

## **GRU**

### Load Model, loss, optimizer, metric

In [0]:
gru_model = GRU()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### Training

In [7]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(gru_model, seqs, labels, optimizer, loss_object, train_loss, train_accuracy)

    for test_seqs, test_labels in test_ds:
        test_step(gru_model, test_seqs, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {} Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(
          epoch + 1,
          train_loss.result(),
          train_accuracy.result()*100,
          test_loss.result(),
          test_accuracy.result()*100
    ))

    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1 Loss: 0.5079026222229004, Accuracy: 73.63999938964844, Test Loss: 0.4441724121570587, Test Accuracy: 78.97599792480469
Epoch 2 Loss: 0.37264707684516907, Accuracy: 83.36799621582031, Test Loss: 0.45184049010276794, Test Accuracy: 78.98400115966797
Epoch 3 Loss: 0.3242509663105011, Accuracy: 86.1520004272461, Test Loss: 0.4839935004711151, Test Accuracy: 78.052001953125
Epoch 4 Loss: 0.2758696973323822, Accuracy: 88.52799987792969, Test Loss: 0.4964955449104309, Test Accuracy: 77.46800231933594
Epoch 5 Loss: 0.23409681022167206, Accuracy: 90.5999984741211, Test Loss: 0.6103373765945435, Test Accuracy: 77.0719985961914
Epoch 6 Loss: 0.19673731923103333, Accuracy: 92.62800598144531, Test Loss: 0.6658809781074524, Test Accuracy: 76.56000518798828
Epoch 7 Loss: 0.16458949446678162, Accuracy: 93.97600555419922, Test Loss: 0.7438694834709167, Test Accuracy: 75.94400024414062
Epoch 8 Loss: 0.13424459099769592, Accuracy: 95.22000122070312, Test Loss: 0.8605367541313171, Test Accuracy: 7